## 配置环境

In [6]:
!pip install grpcio tensorflow-serving-api

  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl.metadata (1.8 kB)
Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)


In [9]:
!pip install keras-image-helper

In [11]:
import grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from keras_image_helper import create_preprocessor

# 设置HTTPS服务证书
import os
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

## 创建连接TF Serving频道
- 调用docker image创建tf serving container
- Container创建成功后，自动生成port

In [3]:
#TF Serving所在port
host = 'localhost:8500'

channel = grpc.insecure_channel(host)

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

## 处理Image Data

In [12]:
preprocessor = create_preprocessor('xception', target_size=(299,299))

In [18]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [22]:
# Conver X format from numpy array to protobuf
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

## 连接TF Serving以及其内部的Clothing Model

In [24]:
pb_request = predict_pb2.PredictRequest()

# saved_model_cli show --dir clothing-model --all
# 从上面bash command中导出的，clothing model内含有的parameters

# 定义Model的名称
pb_request.model_spec.name = 'clothing-model'
# 定义Model的function
pb_request.model_spec.signature_name = 'serving_default'
# 导入数据
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [25]:
#发送数据到port
pb_response = stub.Predict(pb_request, timeout=20.0)

In [34]:
#提取预测结果
preds = pb_response.outputs['dense_7'].float_val

In [35]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [39]:
dict(zip(classes, preds))

{'dress': -1.8798640966415405,
 'hat': -4.75631046295166,
 'longsleeve': -2.359531879425049,
 'outwear': -1.0892632007598877,
 'pants': 9.90378189086914,
 'shirt': -2.8261773586273193,
 'shoes': -3.6483097076416016,
 'shorts': 3.241151809692383,
 'skirt': -2.6120948791503906,
 't-shirt': -4.852035999298096}